In [1]:
"""
Developed by Miles Danswan
Date: 13/04/21
Project: ReOpen A2I2
"""
import requests as rq
import re
import urllib as u
import random as rand
import threading as t
from html.parser import HTMLParser

In [2]:
"""
Simple class inheriting from the Python threading module

Functions as a background task for downloading web scrapping results
"""
class DownloadingThread(t.Thread):
    def __init__(this, batch):
        t.Thread.__init__(this)
        this.results = dict()
        this.batch = batch
    def run(this):
        for url in this.batch:
            try:
                fn = url.rsplit("/", 1)[1]
                path = "./Temp/" + fn
                print("Downloading %s into %s" % (fn[0:10] + "...", path))
                r = rq.get(url)
                with open(path, mode="wb") as file:
                    file.write(r.content)
                this.results[url] = True
            except Exception as e:
                print("Error downloading file from %s\n\t%s" % (url, repr(e)[0:100] + "..."))
                this.results[url] = False

In [3]:
"""
Core HTML analysis and breakdown from the designated image source

Inheriting from HTMLParser allows for simple traversal of the DOM to find embedded image URLs
"""
class CommercialFloorPlanHTMLParser(HTMLParser):
    def __init__(this):
        super().__init__()
        this.image_urls = set()
        this.total_urls = 0
    def handle_starttag(this, tag, attrs):
        if tag == "a":
            if attrs[0][0] == "class" and attrs[0][1] == "serp-item__link":
                img_url_idx = attrs[1][1].find("img_url") + len("img_url") + 1
                img_url_close_idx = attrs[1][1].find("&", img_url_idx)
                this.image_urls.add(u.parse.unquote(attrs[1][1][img_url_idx:img_url_close_idx]))
                this.total_urls += 1

"""
Core class for the image web scrapping process

Simply acts as a manager for the above tasks
"""
class FloorplanDownloader():
    THREADS = 5
    def __init__(this, floorplan_urls):
        this.fp_urls = floorplan_urls
    def download(this):
        if this.fp_urls == None or len(this.fp_urls) == 0:
            print("No URL's Passed")
            return
        rand.shuffle(this.fp_urls)
        size = int(len(this.fp_urls))
        part = int(size / this.THREADS)
        threads = []
        for idx in range(this.THREADS):
            thread = DownloadingThread(this.fp_urls[idx*part:idx*part + part])
            thread.start()
            threads.append(thread)
        for thread in threads:
            thread.join()
        for thread in threads:
            print("Total Potential Images: %d" % (size))

In [4]:
PAGES = 30 # how many pages are to be search in the image engine
parser = CommercialFloorPlanHTMLParser()
for i in range(PAGES):
    url = r'https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=' + str(i) # example url
    print("Acquiring: %s\r" % (url))
    r = rq.get(url, allow_redirects=False)
    parser.feed(str(r.content))
downloader = FloorplanDownloader(list(parser.image_urls))
downloader.download()
parser.close()

Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=0
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=1
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=2
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=3
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=4
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=5
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=6
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=7
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=8
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=9
Acquiring: https://yandex.com/images/search?text=school%20floor%20plans&rpt=image&p=10
Acquiring: https://yandex.com/images/search?text=scho

Error downloading file from https://www.openingthebook.com/media/1039/customer-flow.jpg?width=1600&height=900&mode=crop&anchor=center
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/5007/025c/28ba/0d41/4800/09e5/large_jpg/stringio.jpg?1414442368
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/55e6/ea5b/4d8d/5dd1/7300/1201/large_jpg/rdc-2.jpg?1441196627
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/56fc/8408/e58e/ce3b/c500/00a8/large_jpg/Plan2.jpg?1459389440
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/530d/e40f/c07a/80ed/3b00/00f2/large_jpg/1_plan.jpg?1393419255
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/5525/ca81/e58e/cea1/1900/02c9/large_jpg/floor_(2).jpg?1428540020
	OSError(22, 'Invalid argument')...
Er

Error downloading file from https://images.adsttc.com/media/images/5279/6d34/e8e4/4e86/5400/0038/large_jpg/17_floor_1.jpg?1383689506
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/53b3/5f27/c07a/8079/0f00/0253/large_jpg/9_2nd_floor_plan.jpg?1404264204
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/5014/1a20/28ba/0d3b/4500/1137/large_jpg/stringio.jpg?1414468183
	OSError(22, 'Invalid argument')...
Error downloading file from https://www.lsuagcenter.com/~/media/system/b/3/2/6/b3260baabd5e545e39416c3e51376609/anna1.jpg
	ConnectionError(MaxRetryError("HTTPSConnectionPool(host='www.lsuagcenter.com', port=443): Max retrie...
Error downloading file from https://dailywriting.xyz/wp-content/uploads/2015/09/school-floor-plans-dwg.jpg
	ConnectionError(MaxRetryError("HTTPSConnectionPool(host='dailywriting.xyz', port=443): Max retries e...
Error downloading file from https://images.adsttc.

Error downloading file from https://images.adsttc.com/media/images/5467/9396/e58e/ce12/6900/02f7/large_jpg/floor_(1).jpg?1416074113
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/5631/5c19/e58e/ce22/ae00/053a/large_jpg/TLV_school_-_Ground_Floor.jpg?1446075380
	OSError(22, 'Invalid argument')...
Error downloading file from https://www.metalocus.es/sites/default/files/styles/mopis_news_carousel_item_desktop/public/file-images/metalocus_cobe-kindergarten_23_1180.png?itok=o8kB6gIW
	OSError(22, 'Invalid argument')...
Error downloading file from https://i2.wp.com/www10.aeccafe.com/blogs/arch-showcase/files/2011/04/Brede_school-Tesselseveld_HVE_ground-floor-plan.jpg?w=1000&ssl=1
	OSError(22, 'Invalid argument')...
Error downloading file from https://www.metalocus.es/sites/default/files/styles/mopis_news_carousel_item_desktop/public/metalocus_toho-school_22.jpg?itok=cBTkbKCj
	OSError(22, 'Invalid argument')...
Error downloading file from 

Error downloading file from https://images.adsttc.com/media/images/551e/ba11/e58e/ce84/5e00/01a6/large_jpg/floor_(5).jpg?1428077049
	OSError(22, 'Invalid argument')...
Error downloading file from https://archinect.imgix.net/uploads/bd/bdwzd53383lnmvwn.png?fit=crop&auto=compress%2Cformat&w=615&dpr=3
	OSError(22, 'Invalid argument')...
Error downloading file from https://s3.us-east-1.amazonaws.com/media.archnet.org/system/media_contents/contents/76851/original/IMG42768.jpg?1384735577
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/5605/4619/e58e/ce79/a500/0249/large_jpg/floor.jpg?1443186193
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/556d/1fa5/e58e/ceec/9100/0122/large_jpg/02_Plan_1F.jpg?1433214875
	OSError(22, 'Invalid argument')...
Error downloading file from https://images.adsttc.com/media/images/55e6/1ce1/2347/5db0/4d00/00ec/large_jpg/506db67428ba0d63d4000013_niemenranta-

{'https://i0.wp.com/www.kellybraundesign.com/wp-content/uploads/2011/06/8-high-school-floor-plan.jpg': True, 'https://s3-eu-west-1.amazonaws.com/emap-nibiru-prod/wp-content/uploads/sites/4/2017/12/20111954/freemansswimmingpool_lowergroundfloorplan2.jpg': True, 'https://www.wallpapermatte.com/wp-content/uploads/2018/01/floor-plans-for-school-buildings-10.jpg': True, 'https://www.sl.nsw.gov.au/sites/default/files/4921_mitchell_building_master_plan_floorplan_map_display_910x910mm_update_sept_2017_aw1.jpg': True, 'https://i.pinimg.com/originals/d9/84/6b/d9846bc58ce1f94bdffee3dcced44fd4.jpg': True, 'http://www.designshare.com/dbadmin/upload/projects/1/519/plan%20%5Blevel%202%5D-01.jpg': True, 'https://2cajz746q3h41k3jlsgax8tv-wpengine.netdna-ssl.com/wp-content/uploads/2009/07/0712D001_First_Floor_Plan_monograph-1570x1200.jpg': True, 'https://images.adsttc.com/media/images/518d/1a63/b3fc/4bfe/1500/0071/large_jpg/Augustus_Hawkins_Site_Plan.jpg?1368201809': False, 'https://divisare-res.cloudin